In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

from src.entities import GSA

from typing import Union

In [2]:
# Define corridor

corridor = {"MAD": {
                "CIU": {
                    "COR": {
                        "SEV": {
                            "CAD": {}
                        },
                        "PGE": {
                            "ANT": {
                                "GRA": {},
                                "MAL": {}
                                    }
                                }
                            }
                        }
                    }
            }

In [3]:
# Get lines from corridor

def get_lines(corridor: dict, path: Union[list, None]=None) -> list:
    """
    Get all the lines in the corridor
    
    Args:
        corridor (dict): dictionary with the corridor structure
        path (list, optional): list of nodes
    
    Returns:
        list of lines
    """
    if path is None:
        path = []

    lines = []
    for node, child in corridor.items():
        new_path = path + [node]
        if not child:  # If the node has no children, it is a leaf
            lines.append(new_path)
        else:
            lines.extend(get_lines(child, new_path))  # If the node has children, we call the function recursively

    return lines

get_lines(corridor)

[['MAD', 'CIU', 'COR', 'SEV', 'CAD'],
 ['MAD', 'CIU', 'COR', 'PGE', 'ANT', 'GRA'],
 ['MAD', 'CIU', 'COR', 'PGE', 'ANT', 'MAL']]

In [4]:
def sample_line(lines: list) -> list:
    """
    Sample a random line from the list of lines
    
    Args:
        lines (list): list of lines
    
    Returns:
        list: random line
    """
    return lines[np.random.randint(len(lines))] 

def sample_route(line: list) -> list:
    """
    Sample a random route from line
    
    Args:
        line (list): list of stations
        
    Returns:
        list: random route
    """
    return line[np.random.randint(0, len(line)-1):]

lines = get_lines(corridor)

line = sample_line(lines)
print(f"Sampled line: {line}")

# Sample a random route from line (at least two stations)
route = sample_route(line)
print(f"Sampled route: {route}")

Sampled line: ['MAD', 'CIU', 'COR', 'SEV', 'CAD']
Sampled route: ['MAD', 'CIU', 'COR', 'SEV', 'CAD']


In [5]:
def get_timetable(route: list) -> dict:
    """
    Generate random timetable for route r
    
    Args:
        route (list): list of stations
    
    Returns:
        dict: timetable
    """
    timetable = {}
    AT = np.random.randint(0, 24*60)
    DT = AT
    for i, sta in enumerate(route):
        if i == 0 or i == len(route)-1:
            timetable[sta] = (AT, AT)
        else:
            timetable[sta] = (AT, DT)
            
        AT += np.random.randint(30, 120)
        DT = AT + np.random.randint(2, 8)
        
    return timetable

get_timetable(route)

{'MAD': (773, 773),
 'CIU': (851, 853),
 'COR': (925, 930),
 'SEV': (1017, 1023),
 'CAD': (1092, 1092)}

In [6]:
# Generate random requested timetable in corridor for a day t

# Number of services:
n_services = 100

# 1) Generate random timetable
schedule = {i: get_timetable(sample_route(sample_line(lines))) for i in range(1, n_services+1)}
schedule

{1: {'PGE': (265, 265), 'ANT': (300, 304), 'GRA': (411, 411)},
 2: {'ANT': (353, 353), 'GRA': (424, 424)},
 3: {'ANT': (73, 73), 'MAL': (163, 163)},
 4: {'MAD': (760, 760),
  'CIU': (814, 819),
  'COR': (924, 928),
  'PGE': (996, 1001),
  'ANT': (1081, 1087),
  'GRA': (1128, 1128)},
 5: {'PGE': (665, 665), 'ANT': (747, 749), 'MAL': (860, 860)},
 6: {'MAD': (44, 44),
  'CIU': (149, 155),
  'COR': (251, 254),
  'SEV': (325, 332),
  'CAD': (386, 386)},
 7: {'COR': (740, 740),
  'PGE': (848, 851),
  'ANT': (942, 948),
  'MAL': (1037, 1037)},
 8: {'MAD': (782, 782),
  'CIU': (862, 868),
  'COR': (969, 975),
  'SEV': (1064, 1071),
  'CAD': (1101, 1101)},
 9: {'COR': (723, 723), 'SEV': (800, 804), 'CAD': (875, 875)},
 10: {'MAD': (264, 264),
  'CIU': (379, 384),
  'COR': (446, 453),
  'PGE': (490, 497),
  'ANT': (582, 587),
  'GRA': (659, 659)},
 11: {'COR': (1437, 1437),
  'PGE': (1468, 1475),
  'ANT': (1536, 1539),
  'MAL': (1603, 1603)},
 12: {'MAD': (805, 805),
  'CIU': (917, 920),
  'COR

In [7]:
def is_feasible(solution, security_gap=10):
    """
    Check if the solution is feasible
    
    Args:
        solution (dict): solution
        security_gap (int, optional): security gap
        
    Returns:
        bool: True if the solution is feasible, False otherwise
    """
    solution = np.array(solution["discrete"])
    security_array = []
    
    # Get conflicts between services
    for service in schedule:
        service_sec_arr = []  # Build security matrix for service (columns are stops, rows are other services)
        for service_k in schedule:
          service_sec_row = [] 
          for stop in schedule[service]:
            if service_k == service or stop not in schedule[service_k]:
              service_sec_row.append(0)
              continue
        
            if abs(schedule[service][stop][1] - schedule[service_k][stop][1]) < security_gap:
              service_sec_row.append(1)
            else:
              service_sec_row.append(0)
          service_sec_arr.append(service_sec_row)
        
        security_array.append(np.array(service_sec_arr))
    
    # Get array of security conflicts. If there is a conflict, the dot product will be different from zero
    if not solution.dot(np.array([np.sum(solution.dot(ssa)) for ssa in security_array])):
        return True
    return False

"""
schedule = {1: {'MAD': (0, 0), 'BAR': (148, 152), 'FIG': (180, 180)},
            2: {'MAD': (8, 8), 'ZAR': (28, 30), 'BAR': (165, 167), 'FIG': (210, 210)},
            3: {'MAD': (30, 30), 'BAR': (180, 182), 'FIG': (225, 225)}}

solution = {"discrete": [1, 0, 1]}

is_feasible(solution)
"""

'\nschedule = {1: {\'MAD\': (0, 0), \'BAR\': (148, 152), \'FIG\': (180, 180)},\n            2: {\'MAD\': (8, 8), \'ZAR\': (28, 30), \'BAR\': (165, 167), \'FIG\': (210, 210)},\n            3: {\'MAD\': (30, 30), \'BAR\': (180, 182), \'FIG\': (225, 225)}}\n\nsolution = {"discrete": [1, 0, 1]}\n\nis_feasible(solution)\n'

In [8]:
def get_num_trains(solution):
    scheduled_trains = np.sum(solution["discrete"])
    print("Requested number of trains: ", len(solution["discrete"]))
    print("Scheduled trains: ", scheduled_trains)
    return scheduled_trains, 0

In [9]:
boundaries = {'real': [], 'discrete': [(0, 1) for _ in range(len(schedule))]}

gsa_algo = GSA(objective_function=get_num_trains,
               is_feasible=is_feasible,
               r_dim=0,
               d_dim=len(schedule),
               boundaries=boundaries)

In [10]:
gsa_algo.set_seed(seed=28)

training_history = gsa_algo.optimize(population_size=5,
                                     iters=10,
                                     chaotic_constant=False,
                                     repair_solution=False)

Initializing positions of the individuals in the population...
Positions of the individuals in the population successfully initialized!!
GSA is optimizing  "get_num_trains"
Requested number of trains:  100
Scheduled trains:  6
Requested number of trains:  100
Scheduled trains:  12
Requested number of trains:  100
Scheduled trains:  8
Requested number of trains:  100
Scheduled trains:  6
Requested number of trains:  100
Scheduled trains:  6
['At iteration 1 the best fitness is 12']
Requested number of trains:  100
Scheduled trains:  15
Requested number of trains:  100
Scheduled trains:  4
Requested number of trains:  100
Scheduled trains:  12
Requested number of trains:  100
Scheduled trains:  16
Requested number of trains:  100
Scheduled trains:  12
['At iteration 2 the best fitness is 16']
Requested number of trains:  100
Scheduled trains:  15
Requested number of trains:  100
Scheduled trains:  17
Requested number of trains:  100
Scheduled trains:  8
Requested number of trains:  100
S